In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(123)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import itertools

#import keras
from keras.utils.np_utils import to_categorical # used for converting labels to one-hot-encoding

from tensorflow import keras
from tensorflow.python.keras import backend as k
#from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D



import itertools
from keras.layers.normalization import BatchNormalization

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:

lesion_type_dict = {
0 : "Basal cell carcinoma",
5 : "Vascular lesions", 
2 : "Dermatofibroma",
4 : "Melanoma",
3 : "Melanocytic nevi",
1 : "Benign keratosis-like lesions"}


In [3]:
input_shape = (50, 50, 3)
num_classes = 7
cnnmodel_model = Sequential()
cnnmodel_model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
cnnmodel_model.add(Conv2D(32,kernel_size=(3, 3), activation='relu',padding = 'Same',))
cnnmodel_model.add(MaxPool2D(pool_size = (2, 2)))
cnnmodel_model.add(Dropout(0.25))

cnnmodel_model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
cnnmodel_model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
cnnmodel_model.add(MaxPool2D(pool_size=(2, 2)))
cnnmodel_model.add(Dropout(0.40))

cnnmodel_model.add(Flatten())
cnnmodel_model.add(Dense(128, activation='relu'))
cnnmodel_model.add(Dropout(0.5))
cnnmodel_model.add(Dense(num_classes, activation='softmax'))

cnnmodel_model.load_weights("model_weights.h5")
print("Loaded model from disk")

ValueError: Shapes (9216, 128) and (28800, 128) are incompatible

In [61]:
path_dir = "testdata"
images_path = os.listdir(path_dir)

In [62]:
images = []
images_name = []
for x in images_path:
    images.append(np.asarray(Image.open(os.path.join(path_dir, x)).resize((100,75))))
    images_name.append(x)
    
print(images_name)
images = np.array(images)

['ISIC_0029306.jpg', 'ISIC_0029307.jpg', 'ISIC_0029308.jpg', 'ISIC_0029309.jpg', 'ISIC_0029310.jpg']


In [63]:

testData = images.reshape(images.shape[0], *(75, 100, 3))


In [64]:
#read data for prediction

# Predict the values from the validation dataset
Y_pred = model.predict(testData)
# Convert predictions classes to one hot vectors 

print(Y_pred)

[[0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]]


In [65]:
from numpy import argmax

Y_pred = argmax(Y_pred, axis=1)

In [66]:

images_name = np.array(images_name)
for i in range(len(images_name)):
    print(images_name[i]+"-"+lesion_type_dict.get(Y_pred[i]))

ISIC_0029306.jpg-Melanocytic nevi
ISIC_0029307.jpg-Melanocytic nevi
ISIC_0029308.jpg-Melanocytic nevi
ISIC_0029309.jpg-Melanocytic nevi
ISIC_0029310.jpg-Melanocytic nevi


In [67]:
# Testing

from keras.preprocessing.image import ImageDataGenerator
input_path = "input";

test_data_generator = ImageDataGenerator(rescale=1./255)

test_generator = test_data_generator.flow_from_directory(
    input_path,
    target_size=(50, 50),
    batch_size=1,
    shuffle=False)

test_generator.__len__()

Found 10015 images belonging to 2 classes.


10015

In [68]:

probabilities = model.predict_generator(testData)
probabilities

ValueError: When passing input data as arrays, do not specify `steps_per_epoch`/`steps` argument. Please use `batch_size` instead.

In [70]:
data_path = 'testdata'

In [72]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
probabilities = cnnmodel_model.predict_generator(test_generator,steps=25)

for index, probability in enumerate(probabilities):
    image_path = data_path + "\\" +test_generator.filenames[index]
   
    
    for i, val in enumerate( probability):
        if(val > 0.5):
            print("%.2f" % (probability[i]*100) + "% " + classes[i] + " file:"+test_generator.filenames[index])
            plt.title("%.2f" % (probability[i]*100) + "% " + classes[i] + " file:"+test_generator.filenames[index])
        

    

In [73]:
print(probabilities)

[[0.14178422 0.14361575 0.14567022 0.14313628 0.14132929 0.14311682
  0.14134736]
 [0.14178762 0.14375292 0.14493522 0.14256693 0.14174132 0.14315502
  0.14206101]
 [0.14286073 0.14362988 0.14373128 0.141991   0.1426491  0.14400387
  0.14113416]
 [0.14222372 0.14277807 0.14507504 0.14243011 0.1416951  0.14347702
  0.14232087]
 [0.14357482 0.14276044 0.14400432 0.14233562 0.14149694 0.14372188
  0.14210595]
 [0.14298086 0.14292565 0.14397469 0.14253187 0.14286472 0.14346083
  0.14126141]
 [0.14315537 0.14305574 0.14392665 0.1414331  0.14337996 0.14338131
  0.14166784]
 [0.14316948 0.14344275 0.14348906 0.14115183 0.14279598 0.14381
  0.14214092]
 [0.14323941 0.14282705 0.14254011 0.14296716 0.14200516 0.14407647
  0.14234467]
 [0.14260276 0.14429785 0.14415392 0.14245173 0.14203413 0.14351764
  0.14094192]
 [0.14200902 0.143736   0.14512087 0.14282902 0.14152521 0.14372344
  0.14105646]
 [0.14127994 0.14526685 0.14170225 0.1428028  0.14080691 0.1449836
  0.1431577 ]
 [0.14266145 0.14417

In [74]:
print(image_path)

testdata\HAM10000_images_part_1\ISIC_0024330.jpg


In [76]:
images_names = []

In [77]:
classes = ["Basal cell carcinoma",
           "Vascular lesions",
           "Dermatofibroma",
           "Melanoma",
           "Melanocytic nevi",
           "Benign keratosis-like lesions"
           ]
def printPredictedValues(images_name,pred):
    for i in range(len(images_name)):
        predicted_class = ""
        for j in range(len(pred[i])):
            if(pred[i][j] > 0.5) :
                predicted_class += " "+classes[j]
        
        print(images_name[i] + " belongs to :" + predicted_class);
        

In [82]:
import glob
import cv2

In [83]:
images_for_test = []
images_names = []

for i in glob.glob(os.path.join(data_path,'*')):
    img = cv2.imread(i)
    #img  = cv2.cvtColor(img , cv2.COLOR_BGR2GRAY) # convert to grayscale
    img = cv2.resize(img,(50,50))  #resize
    images_for_test.append(np.array(img))
    
    images_names.append(i.replace(os.path.join(data_path,''), ""))
images_for_test = np.array(images_for_test)

In [85]:
   
images_for_test = []
images_names = []

for i in glob.glob(os.path.join(data_path,'*')):
    img = cv2.imread(i)
    #img  = cv2.cvtColor(img , cv2.COLOR_BGR2GRAY) # convert to grayscale
    img = cv2.resize(img,(50,50))  #resize
    images_for_test.append(np.array(img))
    
    images_names.append(i.replace(os.path.join(data_path,''), ""))
images_for_test = np.array(images_for_test)

In [86]:
# Testing

from keras.preprocessing.image import ImageDataGenerator

data_path = "testdata";
test_data_generator = ImageDataGenerator(rescale=1./255)

test_generator = test_data_generator.flow_from_directory(
    data_path, 
    target_size=(50, 50),
    batch_size=1,
    shuffle=False)

test_generator.__len__()

Found 0 images belonging to 0 classes.


0

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
probabilities = cnnmodel_model.predict_generator(test_generator,steps=1000)

for index, probability in enumerate(probabilities):
    image_path = data_path + "\\" +test_generator.filenames[index]
   
    for i, val in enumerate( probability):
        if(val > 0.5):
            print("%.2f" % (probability[i]*100) + "% " + classes[i] + " file:"+test_generator.filenames[index])
            plt.title("%.2f" % (probability[i]*100) + "% " + classes[i] + " file:"+test_generator.filenames[index])
        